## Импорт библиотек

In [0]:
!git clone https://github.com/sberbank-ai/ru-gpts
!mkdir models/

In [0]:
%pip install torch==1.4.0
%pip install transformers==3.5.0
import torch
import transformers
from transformers import GPT2LMHeadModel, GPT2Tokenizer

## Обучение модели

In [0]:
#!g1.1
%%time
!export PYTHONPATH=${PYTHONPATH}:/ru-gpts/
!CUDA_VISIBLE_DEVICES=0 python3 ru-gpts/pretrain_transformers.py \
    --output_dir=models/gpt3_small_house \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=../data/train_house.txt \
    --do_eval \
    --evaluate_during_training \
    --eval_data_file=../data/valid_house.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 2 \
    --logging_steps 100 \
    --save_steps 100 \
    --block_size 2048 \
    --overwrite_output_dir

## Результаты обученной модели


In [0]:
#!g1.1
model_path = 'models/gpt3_small_house/checkpoint-2000'


# init model
tok = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)
model.cuda()

In [0]:
#!g1.1
def generate_text(text: str, path_to_model: str='models/gpt3_small_house/checkpoint-3000', max_length: int=50):
    
    tokenizer = GPT2Tokenizer.from_pretrained(path_to_model)
    model = GPT2LMHeadModel.from_pretrained(path_to_model)
    model.cuda()
    inpt = tokenizer.encode(text, return_tensors="pt")
    out = model.generate(inpt.cuda(),
                     max_length=max_length, 
                     repetition_penalty=5.0, 
                     do_sample=True, 
                     top_k=5, 
                     top_p=0.95, 
                     temperature=0.8,
                     stop_token='\n')
    out_decoded = tok.decode(out[0])
    answer = out_decoded[len(text):]
    return answer

## Примеры


In [0]:
#!g1.1
generate_text('выдержит ли подставка напечанная на 3д принтре я станцию?')

In [0]:
#!g1.1
generate("Q: выдержит ли подставка напечанная на 3д принтре я станцию?")

In [0]:
#!g1.1
generate("Q: А как колонку подключить к Интернету?", max_length=50)

In [0]:
#!g1.1
generate("Q: Всем привет, может кто знает можно ли подключить Polaris PWK 1775CGLD к Алисе без дополнительных хабов и т.д.?")

In [0]:
#!g1.1
generate("Q: Всем привет, может кто знает можно ли подключить Polaris PWK 1775CGLD к Алисе без дополнительных хабов и т.д.?", max_length=70)